<a href="https://colab.research.google.com/github/zerobbang/study_colab/blob/main/hongong/ch5_2__CrossValidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교차 검증과 그리드 서치

# 검증 세트

전체 데이터 = 훈련 세트 60% + 테스트 세트 20% + 검증 세트 20%

- 데이터 준비

In [50]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alcohol  6497 non-null   float64
 1   sugar    6497 non-null   float64
 2   pH       6497 non-null   float64
 3   class    6497 non-null   float64
dtypes: float64(4)
memory usage: 203.2 KB


데이터 처리를 위해 numpy 배열로 변환한다.

In [51]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

- 훈련 세트와 테스트 세트 분류
  - 데이터를 train(훈련) 세트와 test(테스트) 세트로 나눈다
  - 다시 train 데이터를 sub(훈련) 세트와 val(검증) 세트로 나눈다.
  - sub와 val로 나눌때 train의 20%를 val로 분류한다.

In [56]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target,random_state = 42)

In [57]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target,test_size = 0.2,random_state = 42)

In [58]:
print(sub_input.shape, val_input.shape)

(3897, 3) (975, 3)


- 모델 학습 및 테스트

In [59]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(sub_input,sub_target)
print(dt.score(sub_input,sub_target))
print(dt.score(val_input, val_target))

0.997947138824737
0.841025641025641


-- 이 또한 과대적합이다. 이를 해결하기 위해서 매개변수를 바꿔 좋은 모델을 찾는다. -> 교차검증

# 교차 검증


- 교차 검증 목표 : 좋은 모델

 여기서 말하는 좋은 모델이란 성능이 좋은 모델을 의미하는 것이 아니라 과대 적합이 아닌 모델, 오차가 작은 모델, 안정적인 모델을 의미한다.

## 교차 검증 함수

- cross_validate() : fit_time, score_time, test_score 키를 가진 딕셔너리를 반환
  - fit_time : 훈련하는 시간
  - score_time : 검증하는 시간
  - defult값은 5 -> 5 - 폴드 교차 검증을 수행

In [16]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01156664, 0.00919127, 0.00933456, 0.00908494, 0.00927401]), 'score_time': array([0.00120854, 0.00109029, 0.00104904, 0.00111794, 0.00111294]), 'test_score': array([0.85333333, 0.85333333, 0.8798768 , 0.85420945, 0.8449692 ])}


- 교차 검증의 최종 점수 : test_score 키의 평균을 구한다.

In [60]:
import numpy as np
print(np.mean(scores['test_score']))

0.8598112397751372


 기본적으로 cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않는다. 

 하지만 지금까지 과정은 교차 검증 함수를 사용하기 전에 전체 데이터를 섞은 후 훈련 데이터를 구했기 때문에 섞을 필요가 없었다.

 만약 교차 검증 시 훈련 세트를 섞으려면 분할기를 사용한다.

- 분할기를 사용하여 훈련 세트를 섞자. (10 - 폴드 교차 검증 사용)
  
cross_validate() 함수는 회귀 모델일 경우 KFold 분할기를 사용한다.

In [61]:
from sklearn.model_selection import StratifiedKFold
splitter = StratifiedKFold(n_splits = 10, shuffle =True, random_state=42)
scores = cross_validate(dt,train_input,train_target,cv = splitter)
print(np.mean(scores['test_score']))

0.8585817315784159


## 하이퍼파라미터 튜닝

- 사용자가 지정해야 하는 파라미터
  - 그리드 서치 : 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행 -> 별도로 cross_validate 사용할 일이 없다.
  - 랜덤 서치 : 매개 변수를 샘플링할 수 있는 확률 분포 객체 전달

-> 자동으로 잡아주는 라이브러리들 등장

### 그리드 서치

- 탐색할 매개변수와 탐색할 값의 리스트 생성

In [62]:
from sklearn.model_selection import GridSearchCV
params = {
    'min_impurity_decrease' : [0.0001,0.0002,0.0003,0.0004,0.0005]
}

- 그리드 서치 객체 생성

In [63]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs=-1)

(n_jops 모든 코어를 사용하여 시간을 줄임)

- 그리드 서치 학습

In [64]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [65]:
dt = gs.best_estimator_
print(dt.score(train_input,train_target))

0.9137931034482759


-- 그리드 서치는 검증 점수가 가장 높은 모델의 매개변수 조합으로 자동으로 다시 모델을 훈련하여 gs 객체의 best_estimator_ 속성에 저장한다.

In [66]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0003}


In [67]:
print(gs.cv_results_['mean_test_score'])

[0.86843111 0.86925267 0.87315179 0.87212531 0.87130627]


In [68]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0003}


### 랜덤서치

- 많은 매개 변수 조건으로 그리드 서치 수행 시간이 오래 걸릴 경우 랜덤 서치를 사용한다.
- 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링 할 수 있도록 확률분포를 준다.

In [34]:
# 균등 분포에서 샘플링 한다.
from scipy.stats import uniform,randint
rgen = randint(0,10)
rgen.rvs(10)

array([8, 7, 5, 1, 9, 4, 8, 1, 3, 0])

- 샘플링 숫자를 늘려 고르게 샘플링 되었는지 확인

In [36]:
np.unique(rgen.rvs(1000),return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([103,  94, 111,  84,  97, 103,  97, 101,  99, 111]))

In [72]:
from sklearn.model_selection import RandomizedSearchCV
params = {
    'min_impurity_decrease' : uniform(0.0001,0.001),
    'max_depth' : randint(20,50)
}

In [74]:
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params, n_iter = 100, n_jobs=-1,random_state = 42)
gs.fit(train_input,train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff9ed84c810>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff9ed84cc90>},
                   random_state=42)

In [75]:
gs.best_params_

{'max_depth': 21, 'min_impurity_decrease': 0.0002818249672071006}